In [1]:
import copy
import datetime as dt
import importlib # needed so that we can reload packages
import matplotlib.pyplot as plt
import os, os.path
import numpy as np
import pandas as pd
import pathlib
import sys
import time
from typing import Union
import warnings
from datetime import datetime
warnings.filterwarnings("ignore")
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

##  IMPORT SISEPUEDE EXAMPLES AND TRANSFORMERS

from sisepuede.manager.sisepuede_examples import SISEPUEDEExamples
from sisepuede.manager.sisepuede_file_structure import SISEPUEDEFileStructure
import sisepuede.core.support_classes as sc
import sisepuede.transformers as trf
import sisepuede.utilities._plotting as spu
import sisepuede.utilities._toolbox as sf

file_struct = SISEPUEDEFileStructure()

matt = file_struct.model_attributes

In [2]:
# Define the current directory and output data path
dir_cur = pathlib.Path(os.getcwd()).parent
output_data = dir_cur.joinpath("ssp_run_output")
transformations_dir = dir_cur.joinpath("transformations")

In [3]:
# Change this line only when you receive a new run
run_folder = "sisepuede_run_2025-08-08T18;45;16.252055"
file_name  = f"{run_folder}.csv"
file_name

'sisepuede_run_2025-08-08T18;45;16.252055.csv'

In [4]:
# Define the run folder and tableau data path
run = output_data.joinpath(run_folder)
tableau = dir_cur.joinpath("Tableau/data")
os.makedirs(tableau, exist_ok=True)

In [5]:
# Define the region
region = "uganda"

In [6]:
# Load the output data
df_out_baseline = pd.read_csv(run.joinpath("df_out_baseline.csv"))
df_out_strategy = pd.read_csv(run.joinpath("df_out_strategy.csv"))

In [7]:
df_out_baseline.head()

,time_period,area_agrc_crops_bevs_and_spices,area_agrc_crops_cereals,area_agrc_crops_fibers,area_agrc_crops_fruits,area_agrc_crops_herbs_and_other_perennial_crops,area_agrc_crops_nuts,area_agrc_crops_other_annual,area_agrc_crops_other_woody_perennial,area_agrc_crops_pulses,...,yield_agrc_fruits_tonne,yield_agrc_herbs_and_other_perennial_crops_tonne,yield_agrc_nuts_tonne,yield_agrc_other_annual_tonne,yield_agrc_other_woody_perennial_tonne,yield_agrc_pulses_tonne,yield_agrc_rice_tonne,yield_agrc_sugar_cane_tonne,yield_agrc_tubers_tonne,yield_agrc_vegetables_and_vines_tonne
0,0,747901.028486,2.386299e+06,183833.867477,1.333131e+06,595.329750,605228.344795,3.433943e+06,4242.254753,1.090824e+06,...,9.829842e+06,1264.067389,461200.281192,3.374662e+06,1558.604070,855742.438638,371284.928361,8.144525e+06,8.463351e+06,2.839233e+06
1,1,748561.556254,2.388407e+06,183996.225021,1.334309e+06,595.855531,605762.867563,3.436975e+06,4246.001408,1.091787e+06,...,9.620107e+06,1293.711556,433829.681713,3.383610e+06,1559.980590,877607.385144,381126.126720,8.222345e+06,8.684834e+06,2.890564e+06
2,2,750426.731475,2.394358e+06,184454.684578,1.337633e+06,597.340212,607272.234269,3.445539e+06,4256.581081,1.094507e+06,...,9.613997e+06,1276.325457,468246.878461,3.444833e+06,1563.867562,884788.956022,393222.130981,8.221929e+06,9.937162e+06,2.852242e+06
3,3,754597.062701,2.407664e+06,185479.750849,1.345067e+06,600.659798,610647.015916,3.464687e+06,4280.236093,1.100590e+06,...,9.648496e+06,1281.829283,454828.746296,4.993742e+06,1572.558411,885726.708009,402554.917590,8.216441e+06,1.076563e+07,2.873331e+06
4,4,759702.879272,2.423955e+06,186734.759160,1.354168e+06,604.724032,614778.825868,3.488130e+06,4309.197378,1.108037e+06,...,9.776126e+06,1313.499972,281642.497839,5.330997e+06,1583.198785,886223.484300,406485.577840,8.373521e+06,9.292748e+06,2.907881e+06


In [8]:
df_out_strategy.head()

,time_period,area_agrc_crops_bevs_and_spices,area_agrc_crops_cereals,area_agrc_crops_fibers,area_agrc_crops_fruits,area_agrc_crops_herbs_and_other_perennial_crops,area_agrc_crops_nuts,area_agrc_crops_other_annual,area_agrc_crops_other_woody_perennial,area_agrc_crops_pulses,...,yield_agrc_fruits_tonne,yield_agrc_herbs_and_other_perennial_crops_tonne,yield_agrc_nuts_tonne,yield_agrc_other_annual_tonne,yield_agrc_other_woody_perennial_tonne,yield_agrc_pulses_tonne,yield_agrc_rice_tonne,yield_agrc_sugar_cane_tonne,yield_agrc_tubers_tonne,yield_agrc_vegetables_and_vines_tonne
0,0,747901.028486,2.386299e+06,183833.867477,1.333131e+06,595.329750,605228.344795,3.433943e+06,4242.254753,1.090824e+06,...,9.829842e+06,1264.067389,461200.281192,3.374662e+06,1558.604070,855742.438638,371284.928361,8.144525e+06,8.463351e+06,2.839233e+06
1,1,748561.556254,2.388407e+06,183996.225021,1.334309e+06,595.855531,605762.867563,3.436975e+06,4246.001408,1.091787e+06,...,9.620107e+06,1293.711556,433829.681713,3.383610e+06,1559.980590,877607.385144,381126.126720,8.222345e+06,8.684834e+06,2.890564e+06
2,2,750426.731475,2.394358e+06,184454.684578,1.337633e+06,597.340212,607272.234269,3.445539e+06,4256.581081,1.094507e+06,...,9.613997e+06,1276.325457,468246.878461,3.444833e+06,1563.867562,884788.956022,393222.130981,8.221929e+06,9.937162e+06,2.852242e+06
3,3,754597.062701,2.407664e+06,185479.750849,1.345067e+06,600.659798,610647.015916,3.464687e+06,4280.236093,1.100590e+06,...,9.648496e+06,1281.829283,454828.746296,4.993742e+06,1572.558411,885726.708009,402554.917590,8.216441e+06,1.076563e+07,2.873331e+06
4,4,759702.879272,2.423955e+06,186734.759160,1.354168e+06,604.724032,614778.825868,3.488130e+06,4309.197378,1.108037e+06,...,9.776126e+06,1313.499972,281642.497839,5.330997e+06,1583.198785,886223.484300,406485.577840,8.373521e+06,9.292748e+06,2.907881e+06


In [9]:
# manually add strategy_id to both dfs
df_out_baseline["strategy_id"] = 0
df_out_strategy["strategy_id"] = 6006

# manually add primary_id to both dfs
df_out_baseline["primary_id"] = 0
df_out_strategy["primary_id"] = 69069

In [10]:
# concat both dataframes
ssp_output = pd.concat([df_out_baseline, df_out_strategy], ignore_index=True)
ssp_output.head()

,time_period,area_agrc_crops_bevs_and_spices,area_agrc_crops_cereals,area_agrc_crops_fibers,area_agrc_crops_fruits,area_agrc_crops_herbs_and_other_perennial_crops,area_agrc_crops_nuts,area_agrc_crops_other_annual,area_agrc_crops_other_woody_perennial,area_agrc_crops_pulses,...,yield_agrc_nuts_tonne,yield_agrc_other_annual_tonne,yield_agrc_other_woody_perennial_tonne,yield_agrc_pulses_tonne,yield_agrc_rice_tonne,yield_agrc_sugar_cane_tonne,yield_agrc_tubers_tonne,yield_agrc_vegetables_and_vines_tonne,strategy_id,primary_id
0,0,747901.028486,2.386299e+06,183833.867477,1.333131e+06,595.329750,605228.344795,3.433943e+06,4242.254753,1.090824e+06,...,461200.281192,3.374662e+06,1558.604070,855742.438638,371284.928361,8.144525e+06,8.463351e+06,2.839233e+06,0,0
1,1,748561.556254,2.388407e+06,183996.225021,1.334309e+06,595.855531,605762.867563,3.436975e+06,4246.001408,1.091787e+06,...,433829.681713,3.383610e+06,1559.980590,877607.385144,381126.126720,8.222345e+06,8.684834e+06,2.890564e+06,0,0
2,2,750426.731475,2.394358e+06,184454.684578,1.337633e+06,597.340212,607272.234269,3.445539e+06,4256.581081,1.094507e+06,...,468246.878461,3.444833e+06,1563.867562,884788.956022,393222.130981,8.221929e+06,9.937162e+06,2.852242e+06,0,0
3,3,754597.062701,2.407664e+06,185479.750849,1.345067e+06,600.659798,610647.015916,3.464687e+06,4280.236093,1.100590e+06,...,454828.746296,4.993742e+06,1572.558411,885726.708009,402554.917590,8.216441e+06,1.076563e+07,2.873331e+06,0,0
4,4,759702.879272,2.423955e+06,186734.759160,1.354168e+06,604.724032,614778.825868,3.488130e+06,4309.197378,1.108037e+06,...,281642.497839,5.330997e+06,1583.198785,886223.484300,406485.577840,8.373521e+06,9.292748e+06,2.907881e+06,0,0


In [11]:
ssp_output.tail()

,time_period,area_agrc_crops_bevs_and_spices,area_agrc_crops_cereals,area_agrc_crops_fibers,area_agrc_crops_fruits,area_agrc_crops_herbs_and_other_perennial_crops,area_agrc_crops_nuts,area_agrc_crops_other_annual,area_agrc_crops_other_woody_perennial,area_agrc_crops_pulses,...,yield_agrc_nuts_tonne,yield_agrc_other_annual_tonne,yield_agrc_other_woody_perennial_tonne,yield_agrc_pulses_tonne,yield_agrc_rice_tonne,yield_agrc_sugar_cane_tonne,yield_agrc_tubers_tonne,yield_agrc_vegetables_and_vines_tonne,strategy_id,primary_id
27,11,780797.175503,2.491260e+06,191919.731382,1.391769e+06,621.515107,631849.087180,3.584983e+06,4428.848742,1.138803e+06,...,283634.468395,1.083833e+07,1822.417729,1.033178e+06,487562.324986,9.580337e+06,1.096037e+07,3.342942e+06,6006,69069
28,12,783258.976442,2.499115e+06,192524.841377,1.396157e+06,623.474702,633841.264823,3.596286e+06,4442.812604,1.142394e+06,...,289609.619296,1.106665e+07,1860.809470,1.054943e+06,497833.497183,9.782160e+06,1.119127e+07,3.413366e+06,6006,69069
29,13,785682.491048,2.506847e+06,193120.540602,1.400477e+06,625.403821,635802.459790,3.607414e+06,4456.559298,1.145928e+06,...,295602.304073,1.129564e+07,1899.313870,1.076772e+06,508134.809781,9.984576e+06,1.142284e+07,3.483996e+06,6006,69069
30,14,788068.162239,2.514459e+06,193706.937926,1.404729e+06,627.302817,637733.030510,3.618368e+06,4470.091335,1.149408e+06,...,301611.947169,1.152529e+07,1937.927231,1.098663e+06,518465.273413,1.018756e+07,1.165507e+07,3.554826e+06,6006,69069
31,15,790416.311505,2.521951e+06,194284.112371,1.408915e+06,629.171946,639633.237140,3.629149e+06,4483.410541,1.152833e+06,...,307637.937260,1.175555e+07,1976.645625,1.120614e+06,528823.837220,1.039110e+07,1.188793e+07,3.625849e+06,6006,69069


In [12]:
# # Load atribute primary data
# df_primary =pd.read_csv(run.joinpath("ATTRIBUTE_PRIMARY.csv"))
# df_primary.head()

In [13]:
# Load atribute strategy data
df_strategy =pd.read_csv(transformations_dir.joinpath("strategy_definitions.csv"))
df_strategy.tail()

,strategy_id,strategy_code,strategy,description,transformation_specification
69,6002,PFLO:ALL,All Actions,All actions (unique by transformer),TX:AGRC:DEC_CH4_RICE|TX:AGRC:DEC_EXPORTS|TX:AG...
70,6003,PFLO:NDC,NDC,NDC Pathway,TX:LSMM:INC_MANAGEMENT_POULTRY_STRATEGY_NDC|TX...
71,6004,PFLO:NZ,NZ,Custom Strategy,TX:WASO:INC_LANDFILLING_STRATEGY_NZ|TX:LSMM:IN...
72,6005,PFLO:BAU,BAU,Custom Strategy,TX:AGRC:INC_PRODUCTIVITY_STRATEGY_BAU|TX:LVST:...
73,6006,PFLO:NDC_2,NDC_2,Custom Strategy,TX:LVST:INC_PRODUCTIVITY_STRATEGY_NDC_2|TX:WAS...


In [14]:
# Merge the dataframes to include primary and strategy information in the ssp_output
# ssp_output = ssp_output.merge(  df_primary, how='left', on='primary_id')
ssp_output = ssp_output.merge(df_strategy[["strategy_id", "strategy_code"]], how='left', on='strategy_id')
ssp_output.head()

,time_period,area_agrc_crops_bevs_and_spices,area_agrc_crops_cereals,area_agrc_crops_fibers,area_agrc_crops_fruits,area_agrc_crops_herbs_and_other_perennial_crops,area_agrc_crops_nuts,area_agrc_crops_other_annual,area_agrc_crops_other_woody_perennial,area_agrc_crops_pulses,...,yield_agrc_other_annual_tonne,yield_agrc_other_woody_perennial_tonne,yield_agrc_pulses_tonne,yield_agrc_rice_tonne,yield_agrc_sugar_cane_tonne,yield_agrc_tubers_tonne,yield_agrc_vegetables_and_vines_tonne,strategy_id,primary_id,strategy_code
0,0,747901.028486,2.386299e+06,183833.867477,1.333131e+06,595.329750,605228.344795,3.433943e+06,4242.254753,1.090824e+06,...,3.374662e+06,1558.604070,855742.438638,371284.928361,8.144525e+06,8.463351e+06,2.839233e+06,0,0,BASE
1,1,748561.556254,2.388407e+06,183996.225021,1.334309e+06,595.855531,605762.867563,3.436975e+06,4246.001408,1.091787e+06,...,3.383610e+06,1559.980590,877607.385144,381126.126720,8.222345e+06,8.684834e+06,2.890564e+06,0,0,BASE
2,2,750426.731475,2.394358e+06,184454.684578,1.337633e+06,597.340212,607272.234269,3.445539e+06,4256.581081,1.094507e+06,...,3.444833e+06,1563.867562,884788.956022,393222.130981,8.221929e+06,9.937162e+06,2.852242e+06,0,0,BASE
3,3,754597.062701,2.407664e+06,185479.750849,1.345067e+06,600.659798,610647.015916,3.464687e+06,4280.236093,1.100590e+06,...,4.993742e+06,1572.558411,885726.708009,402554.917590,8.216441e+06,1.076563e+07,2.873331e+06,0,0,BASE
4,4,759702.879272,2.423955e+06,186734.759160,1.354168e+06,604.724032,614778.825868,3.488130e+06,4309.197378,1.108037e+06,...,5.330997e+06,1583.198785,886223.484300,406485.577840,8.373521e+06,9.292748e+06,2.907881e+06,0,0,BASE


In [15]:
# Strategys in the output
ssp_output.value_counts('strategy_code')

strategy_code
BASE          16
PFLO:NDC_2    16
Name: count, dtype: int64

In [16]:
# Get the list of all fields in the output
subsector_emission_fields = matt.get_all_subsector_emission_total_fields()
subsector_emission_fields

['emission_co2e_subsector_total_agrc',
 'emission_co2e_subsector_total_frst',
 'emission_co2e_subsector_total_lndu',
 'emission_co2e_subsector_total_lsmm',
 'emission_co2e_subsector_total_lvst',
 'emission_co2e_subsector_total_soil',
 'emission_co2e_subsector_total_waso',
 'emission_co2e_subsector_total_trww',
 'emission_co2e_subsector_total_ccsq',
 'emission_co2e_subsector_total_entc',
 'emission_co2e_subsector_total_fgtv',
 'emission_co2e_subsector_total_inen',
 'emission_co2e_subsector_total_scoe',
 'emission_co2e_subsector_total_trns',
 'emission_co2e_subsector_total_ippu']

In [17]:
# Define subsector emission fields that are not energy related
subsector_emission_fields_non_energy = [f for f in subsector_emission_fields if f not in ['emission_co2e_subsector_total_inen', 'emission_co2e_subsector_total_fgtv', 'emission_co2e_subsector_total_entc']]
subsector_emission_fields_non_energy

['emission_co2e_subsector_total_agrc',
 'emission_co2e_subsector_total_frst',
 'emission_co2e_subsector_total_lndu',
 'emission_co2e_subsector_total_lsmm',
 'emission_co2e_subsector_total_lvst',
 'emission_co2e_subsector_total_soil',
 'emission_co2e_subsector_total_waso',
 'emission_co2e_subsector_total_trww',
 'emission_co2e_subsector_total_ccsq',
 'emission_co2e_subsector_total_scoe',
 'emission_co2e_subsector_total_trns',
 'emission_co2e_subsector_total_ippu']

In [18]:
# Define year based on time_period
ssp_output['year'] = ssp_output['time_period'] + 2015

In [19]:
# Melt ssp_output to long format for variables in subsector_emission_fields
ssp_long = ssp_output.melt(
    id_vars=['primary_id', 'strategy_code', 'strategy_id', 'year'],
    value_vars=subsector_emission_fields,
    var_name='Subsector',
    value_name='Emission'
)
ssp_long.head()

,primary_id,strategy_code,strategy_id,year,Subsector,Emission
0,0,BASE,0,2015,emission_co2e_subsector_total_agrc,29.342414
1,0,BASE,0,2016,emission_co2e_subsector_total_agrc,29.384122
2,0,BASE,0,2017,emission_co2e_subsector_total_agrc,29.506592
3,0,BASE,0,2018,emission_co2e_subsector_total_agrc,29.832551
4,0,BASE,0,2019,emission_co2e_subsector_total_agrc,30.012443


In [20]:
# Clean the 'Subsector' column to remove the prefix and capitalize subsector names
ssp_long['Subsector'] = ssp_long['Subsector'].str.replace('emission_co2e_subsector_total_', '', regex=False)
ssp_long['Subsector'] = ssp_long['Subsector'].str.capitalize()
ssp_long.head()

,primary_id,strategy_code,strategy_id,year,Subsector,Emission
0,0,BASE,0,2015,Agrc,29.342414
1,0,BASE,0,2016,Agrc,29.384122
2,0,BASE,0,2017,Agrc,29.506592
3,0,BASE,0,2018,Agrc,29.832551
4,0,BASE,0,2019,Agrc,30.012443


In [21]:
# List of subsectors in the output
ssp_long.value_counts('Subsector')

Subsector
Agrc    32
Ccsq    32
Entc    32
Fgtv    32
Frst    32
Inen    32
Ippu    32
Lndu    32
Lsmm    32
Lvst    32
Scoe    32
Soil    32
Trns    32
Trww    32
Waso    32
Name: count, dtype: int64

In [22]:
# Define a dictionary to map subsector codes to full names
subsector_dict = {
    "Agrc": "Agriculture",
    "Frst": "Forest",
    "Lndu": "Land Use",
    "Lsmm": "Livestock Manure Management",
    "Lvst": "Livestock",
    "Soil": "Soil Management",
    "Wail": "Liquid Waste",
    "Waso": "Solid Waste",
    "Trww": "Wastewater Treatment",
    "Ccsq": "Carbon Capture and Sequestration",
    "Enfu": "Energy Fuels",
    "Enst": "Energy Storage",
    "Entc": "Energy Technology",
    "Fgtv": "Fugitive Emissions",
    "Inen": "Industrial Energy",
    "Scoe": "Stationary Combustion and Other Energy",
    "Trns": "Transportation",
    "Trde": "Transportation Demand",
    "Ippu": "IPPU",
    "Econ": "Economy",
    "Gnrl": "General"
}

ssp_long['Subsector'] = ssp_long['Subsector'].replace(subsector_dict)
ssp_long.value_counts('Subsector')

Subsector
Agriculture                               32
Carbon Capture and Sequestration          32
Energy Technology                         32
Forest                                    32
Fugitive Emissions                        32
IPPU                                      32
Industrial Energy                         32
Land Use                                  32
Livestock                                 32
Livestock Manure Management               32
Soil Management                           32
Solid Waste                               32
Stationary Combustion and Other Energy    32
Transportation                            32
Wastewater Treatment                      32
Name: count, dtype: int64

In [23]:
# Rename columns
ssp_long = ssp_long.rename(columns={"primary_id": "Primary Id",
                                    "strategy_code": "Strategy",
                                    "strategy_id": "Strategy Id",
                                    "year": "Year"})
ssp_long

,Primary Id,Strategy,Strategy Id,Year,Subsector,Emission
0,0,BASE,0,2015,Agriculture,29.342414
1,0,BASE,0,2016,Agriculture,29.384122
2,0,BASE,0,2017,Agriculture,29.506592
3,0,BASE,0,2018,Agriculture,29.832551
4,0,BASE,0,2019,Agriculture,30.012443
...,...,...,...,...,...,...
475,69069,PFLO:NDC_2,6006,2026,IPPU,3.320737
476,69069,PFLO:NDC_2,6006,2027,IPPU,3.672808
477,69069,PFLO:NDC_2,6006,2028,IPPU,4.079016
478,69069,PFLO:NDC_2,6006,2029,IPPU,4.544950


In [24]:
# Save the long format data to a CSV file for Tableau
ssp_long.to_csv(tableau.joinpath(f"emissions_{region}_{file_name}.csv"), index=False)

In [25]:
ssp_long.columns

Index(['Primary Id', 'Strategy', 'Strategy Id', 'Year', 'Subsector',
       'Emission'],
      dtype='object')

In [26]:
# double check of total emissions for strategy 6006
df_check = ssp_long[(ssp_long['Strategy Id'] == 6006) & (ssp_long['Year'] == 2030)].copy()
df_check

,Primary Id,Strategy,Strategy Id,Year,Subsector,Emission
31,69069,PFLO:NDC_2,6006,2030,Agriculture,31.748745
63,69069,PFLO:NDC_2,6006,2030,Forest,-13.057711
95,69069,PFLO:NDC_2,6006,2030,Land Use,2.633271
127,69069,PFLO:NDC_2,6006,2030,Livestock Manure Management,4.871137
159,69069,PFLO:NDC_2,6006,2030,Livestock,12.270040
191,69069,PFLO:NDC_2,6006,2030,Soil Management,6.571444
223,69069,PFLO:NDC_2,6006,2030,Solid Waste,13.779096
255,69069,PFLO:NDC_2,6006,2030,Wastewater Treatment,10.849549
287,69069,PFLO:NDC_2,6006,2030,Carbon Capture and Sequestration,0.000000
319,69069,PFLO:NDC_2,6006,2030,Energy Technology,4.988059


In [27]:
df_check['Emission'].sum(axis=0)

np.float64(191.56676538567322)